In [1]:
"""
TODO: Enter any documentation that only people updating the metric should read here.

All columns of the solution and submission dataframes are passed to your metric, except for the Usage column.

Your metric must satisfy the following constraints:
- You must have a function named score. Kaggle's evaluation system will call that function.
- You can add your own arguments to score, but you cannot change the first three (solution, submission, and row_id_column_name).
- All arguments for score must have type annotations.
- score must return a single, finite, non-null float.
"""

import pandas as pd
import pandas.api.types


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Computes the accuracy/latency score.
    '''
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    # TODO: adapt or remove this check depending on what data types make sense for your metric
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    
    # Compute metric here
    acc_dict = (solution.iloc[:, 0] ==
            submission.iloc[:, 0]).value_counts().to_dict()

    accuracy = 100*(acc_dict[True] / len(solution))
    latency = submission.iloc[:, 1].mean()
    if latency < 0.01:
        raise Exception(f'Latency should be in milliseconds. Your avg latency is {latency} and appears to be too low.')
    score = accuracy / latency
    return score
